In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D,BatchNormalization,LeakyReLU
from sklearn.metrics import confusion_matrix

# ignore warnings
import warnings
warnings.simplefilter("ignore")



In [ ]:
# !head -n 2 /kaggle/input/digit-recognizer/train.csv
train_data = pd.read_csv("{}/train.csv".format(dirname))
test_data = pd.read_csv("{}/test.csv".format(dirname))

In [ ]:
train_data.head()

In [ ]:
test_data.to_numpy().shape

In [ ]:
train_data.to_numpy().shape

In [ ]:
np.max(train_data.to_numpy()[0])

In [ ]:
from skimage import feature
from skimage import filters
from skimage.transform import resize
# resize(heatmap.numpy(),(28,28),)

train_label = train_data.label.to_numpy()

def ArrayFilter(imgarray):
    img2 = np.array([imgarray,#edge_img,edge_img
                    ]).transpose(1,2,0)
    return img2

train_image=np.array([ 
        ArrayFilter(img.reshape(28,28))
    for img in train_data.to_numpy()[0:,1:]
])

test_image =np.array([
        ArrayFilter(img.reshape(28,28))
    for img in test_data.to_numpy() 
])


In [ ]:
# この関数は、1行5列のグリッド形式で画像をプロットし、画像は各列に配置されます。
def plotImages(images_arr,title_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,10))
    axes = axes.flatten()
    for img, ax , title in zip( images_arr, axes,title_arr):
        ax.set_title(title)
        ax.imshow(img,
                  cmap="gray"
                 )
        ax.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255, 
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    #horizontal_flip=True,
    validation_split=0.2)

In [ ]:
train_data_gen=datagen.flow(train_image,train_label, batch_size=294, shuffle=True,subset="training")
valid_data_gen=datagen.flow(train_image,train_label, 
                                       batch_size=126, subset="validation")

for indx in range(0,3):
    augmented_images = [train_data_gen[indx][0][i] for i in range(10)]
    plotImages(augmented_images,train_data_gen[indx][1][0:10])

In [ ]:
# Hyper parameterのfine tuneを行う
import keras_tuner as kt
import IPython

In [ ]:
print("shape:",train_data_gen[0][0][0].shape)
print("max:",np.max(train_data_gen[0][0][0]))

In [ ]:
train_image[0].shape

In [ ]:

# have completed tune at Version11
def model_builder(hp):
    # input layer
    hp_input_layer = hp.Int("InputParam",min_value=28, max_value=28,step=4)
    # layer1
    hp_drop_rate1 = hp.Choice('drop_rate1', values = [0.2]) 
    hp_layer_units1 = hp.Int('units1', min_value = 28, max_value = 28, step = 4)
    # hp_layer_units1 = hp.Int('units1', min_value = 28, max_value = 36, step = 4)
    hp_reg_rate1 = hp.Choice('reg_rate1', values = [ #1e-2,1e-3,
                                                    1e-4]) 
    # layer2
    hp_layer_units2 = hp.Int('units2', min_value = 28, max_value = 28, step = 4)
    # hp_layer_units2 = hp.Int('units2', min_value = 28, max_value = 36, step = 4)
    hp_reg_rate2 = hp.Choice('reg_rate2', values = [# 1e-2,1e-3,
                                                    1e-4]) 

    model = keras.Sequential([
        keras.Input(shape=(28,28,1)),
        Conv2D(hp_input_layer, (4,4), activation='relu',
               name = "InputLayer"),
        # layer1
        Conv2D(hp_layer_units1, (4,4),activation='relu',
          kernel_regularizer=keras.regularizers.l2(hp_reg_rate1)),
        BatchNormalization(),
        LeakyReLU(0.2),
        keras.layers.Dropout(hp_drop_rate1),

        # layer2
        Conv2D(hp_layer_units2,(4,4) ,activation='relu', padding="same",
               kernel_regularizer=keras.regularizers.l2(hp_reg_rate2),name="layer2"),
        BatchNormalization(),
        LeakyReLU(0.2),
        keras.layers.Dropout(hp_drop_rate1),
        keras.layers.AveragePooling2D(pool_size=(2, 2)),
    ])
    
    """
    hp_layer3_flag = hp.Choice('layer3_flag', values = [True,
                                                        False
                                                    ]) 
    if hp_layer3_flag:
        model.add(
            Conv2D(28,(3,3) , padding="same",name= "layer3",
                   activation='relu',kernel_regularizer=keras.regularizers.l2(1e-4))
        )
        model.add(BatchNormalization())
        model.add(LeakyReLU(0.2))
    """
    
    model.add(
        Conv2D(28,(3,3) , padding="same",name= "layer3",
               activation='relu',kernel_regularizer=keras.regularizers.l2(1e-4))
    )
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))

    model.add(
        Conv2D(28,(3,3) , padding="same",name= "lastConvLayer",
               activation='relu',kernel_regularizer=keras.regularizers.l2(1e-4))
    )
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    # compile 
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2]) 
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  metrics=["accuracy"])
    return model


In [ ]:
!rm -rf ./my_dir
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 50,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", 
                                              factor=0.5, patience=5, 
                                              min_lr=1e-8, verbose=1)

In [ ]:
tuner.search(train_data_gen,
             steps_per_epoch=100, 
             epochs=50, 
             validation_data=valid_data_gen, 
             validation_steps=50, 
             callbacks = [ClearTrainingOutput(),
                          reduce_lr,
                          early_stop]
            )

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

from pprint import pprint
print("tuned model parameter----------")
pprint(best_hps.values)
print("----------")

model = tuner.hypermodel.build(best_hps)
model.summary()

In [ ]:
hist=model.fit(train_data_gen,
               steps_per_epoch=100, 
               epochs=200, 
               validation_data=valid_data_gen, 
               validation_steps=50, 
               callbacks = [reduce_lr,early_stop],
               verbose=2
            )

In [ ]:
results = model.evaluate(valid_data_gen, verbose=2)

In [ ]:
history_dict = hist.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
hist_result = (acc,val_acc),(loss,val_loss) 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,8))
axes = axes.flatten()

for res, ax,title in zip( hist_result, axes,["Accuracy","loss"]):
    ax.plot(epochs,res[0], 'b', label='{} {}'.format("Training",title)) 
    ax.plot(epochs,res[1], 'r', label='{} {}'.format("Validation",title)) 
    ax.set_xlabel('Epochs')
    ax.set_ylabel(title)
    ax.set_xlim(0,epochs[-1])
    ax.grid()
    ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
data = train_data_gen # train_data_gen
results =[ tf.argmax(i) for i in model.predict(data.x, verbose=2)]
confusion_mtx = confusion_matrix(data.y, results) 

import seaborn as sns
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="cubehelix",linecolor="gray", fmt= '.1f')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
confusion_mtx

In [ ]:
res = model.predict(test_image[:]/255)
output = pd.DataFrame({'ImageId':[ i+1 for i in range(len(test_image))], 
                       'Label': [ xi.argmax() for xi in res]})
output.to_csv('submission_grid.csv', index=False)

In [ ]:
offset = 5
for i in range(3):
    print("predicted: ",res[i+ offset].argmax())
    plt.imshow(test_image[i + offset],cmap="gray")
    plt.show()

---

# Try using Grad-cam

- Credit
    - https://keras.io/examples/vision/integrated_gradients/
    - https://keras.io/examples/vision/grad_cam/

In [ ]:
def show_figure(img_arr,alpha=0.5,pred=None):
    # print(pred)
    col = 3
    if pred is not None:
        col = 4
    fig, axes = plt.subplots(1, col, figsize=(20,5))
    axes = axes.flatten()
    axes[0].imshow(img_arr[0],cmap="gray")
    axes[0].imshow(img_arr[1],alpha=alpha)
    axes[1].imshow(img_arr[0],cmap="gray")
    axes[2].imshow(img_arr[1])
    if col == 4:
        x = [i for i in range(len(pred[0]))]
        axes[3].bar(x, pred[0])
        axes[3].set_xticks(x)
        axes[3].set_yscale('log')
        axes[3].set_xlabel("Class")
        axes[3].set_ylabel("Score")

    axes[0].axis('off')
    axes[1].axis('off')
    axes[2].axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
def get_gradients(img_input,index):
    images = tf.cast(img_input/255, tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(images)
        preds = model(images)
        class_channel = preds[:, index]
        index =tf.argmax(preds[0])
        print("index: ",index, " rate:",preds[0][index])
        # preds[0][index] = 0
    grads = tape.gradient(class_channel, images)
    # print(grads.shape)
    glad_img = np.array(grads[0]).reshape(28,28,1)
    glad_img = (glad_img-np.min(glad_img))*255 /(np.max(glad_img)-np.min(glad_img))
    return img_input[0],glad_img

In [ ]:
offset = 20
for i in range(0,10):
    num = i
    pred = model.predict(test_image[i+offset:i+offset+1]/255)
    print("pledicted result",pred[0].argmax())
    img_arr  = get_gradients(test_image[i+offset:i+offset+1],pred[0].argmax())
    show_figure(img_arr,pred=pred)


---

# Grad-cam 

Am I using grad-cam correctly?  
If it can be used, is this model training correctly?  
I would like advice from someone who is familiar with it.

In [ ]:
from skimage.transform import resize


In [ ]:
lastCoveLater = "lastConvLayer"
def make_gradcam_heatmap(img_array, model, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(lastCoveLater).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array/255)
        # print("predicted index:",tf.argmax(preds[0]),"\n result: ",preds)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    #print(last_conv_layer_output.shape)
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    # print(heatmap.shape)
    heatmap = tf.squeeze(heatmap)
    # print(heatmap.shape)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return resize(heatmap.numpy(),(28,28),)

#bottle_resized = resize(bottle, (140, 54), anti_aliasing=True)


In [ ]:
offset = 25000
num = 10
for i in range(offset,offset +num):
    num = i
    pred = model.predict(test_image[num:num+1]/255)
    print("index:",pred[0].argmax(), "labels: ",pred[0])
    heatmap = make_gradcam_heatmap(test_image[num:num+1],model,pred[0].argmax())
    arr = [test_image[num],heatmap]
    show_figure(arr,alpha=0.7,pred=pred)

# try image segmentation

## first  make segmentation data set


In [ ]:
def segImg(imgarray):
    # use edge filter
    edge =filters.sobel(imgarray)
    edge_img = np.array( [edge/np.max(edge)])
    
    edge_img_bn = np.where( edge_img >= 0.5,1,0)
    return edge_img_bn.transpose(1,2,0)


train_edge = np.array([ 
        segImg(img.reshape(28,28))
    for img in train_data.to_numpy()[0:,1:]
])

print(train_edge.shape)
for i in range(0,3):
    plt.imshow(train_edge[i]);
    plt.show()

In [ ]:
def make_model():
    inputs = keras.Input(shape=(28,28,1))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = Conv2D(32, (5,5), padding="same",
               kernel_regularizer=keras.regularizers.l2(1e-4))(inputs)
    x = LeakyReLU(0.2)(x)
    x = BatchNormalization()(x)

    # Conv block
    x = Conv2D(36, (5,5), padding="same",
               kernel_regularizer=keras.regularizers.l2(1e-4))(x)

    x = LeakyReLU(0.2)(x)
    x = BatchNormalization()(x)
    x = keras.layers.AveragePooling2D(pool_size=(2, 2))(x)
    x = keras.layers.Dropout(0.3)(x)
    
    # classification layer
    class_ = Flatten()(x)
    y = Dense(10, activation='softmax',name="class")(class_)

    # segmentaion layer
    segmentaion = keras.layers.UpSampling2D(2)(x)
    segmentaion = Conv2D(2, (4,4), padding="same",activation='softmax',name="segment")(segmentaion)

    model = keras.Model(inputs, [y,segmentaion])
    return model
                         
seg_model = make_model()
seg_model.summary()

res=seg_model(train_data_gen[0][0][0:1])

In [ ]:
keras.utils.plot_model(seg_model, "my_model.png", show_shapes=True)

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='class_loss', patience=20)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_class_accuracy', 
                                              factor=0.5, patience=10, 
                                              min_lr=1e-8, verbose=1)

seg_model.compile(loss=[ "sparse_categorical_crossentropy" ,"sparse_categorical_crossentropy"],
              loss_weights = [1.0,0.3],
              optimizer = keras.optimizers.Adam(learning_rate = 0.001),
              metrics=["accuracy"])


hist=seg_model.fit(train_image/255,
                   {"class":train_label,"segment":train_edge},
                   steps_per_epoch=200, 
                   batch_size=400,
                   validation_batch_size=200,
                   validation_split=0.2,
                   epochs=200,
                   shuffle=True,
                   callbacks = [reduce_lr,early_stop],
                   verbose=2
            )

In [ ]:
history_dict = hist.history

acc = history_dict['class_accuracy']
val_acc = history_dict['val_class_accuracy']
loss = history_dict['class_loss']
val_loss = history_dict['val_class_loss']

epochs = range(1, len(acc) + 1)
hist_result = (acc,val_acc),(loss,val_loss) 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20,8))
axes = axes.flatten()

for res, ax,title in zip( hist_result, axes,["Accuracy","loss"]):
    ax.plot(epochs,res[0], 'b', label='{} {}'.format("Training",title)) 
    ax.plot(epochs,res[1], 'r', label='{} {}'.format("Validation",title)) 
    ax.set_xlabel('Epochs')
    ax.set_ylabel(title)
    ax.set_xlim(2,epochs[-1])
    #ax.set_ylim(0.97,1.01)
    ax.grid()
    ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
data = train_image/255 # train_data_gen
results =[ tf.argmax(i) for i in seg_model.predict(data)[0]]
confusion_mtx = confusion_matrix(train_label, results) 

import seaborn as sns
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="cubehelix",linecolor="gray", fmt= '.1f')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
res = seg_model.predict(test_image[:]/255)[0]
output = pd.DataFrame({'ImageId':[ i+1 for i in range(len(test_image))], 
                       'Label': [ xi.argmax() for xi in res]})
output.to_csv('submission_grid2.csv', index=False)

In [ ]:
!head ./submission_grid2.csv

In [ ]:
num =0
for num in range(num,num+80):
    fig, axes = plt.subplots(1, 3, figsize=(18,6))
    axes = axes.flatten()

    img = test_image[num:num+1]/255
    res=seg_model.predict(img)

    print(tf.argmax(res[0][0]))

    axes[0].imshow(img[0])

    im=axes[1].imshow(res[1][0][:,:,1])
    fig.colorbar(im)

    x = [i for i in range(10)]
    axes[2].bar(x, res[0][0])
    axes[2].set_xticks(x)
    axes[2].set_yscale('log')
    axes[2].set_xlabel("Class")
    axes[2].set_ylabel("Score")
    axes[2].set_ylim(1e-2,1)
    plt.show()